# NBA Team Game Logs – Data Pipeline

Dieses Notebook erstellt einen konsistenten Master-Datensatz auf **Team × Spiel-Ebene** für mehrere NBA-Saisons.  
Die Daten stammen aus der offiziellen NBA API (`nba_api`) und werden für spätere Analysen (z. B. Three-Point-Trends) vorbereitet.


## Zentrales Spalten-Mapping

In diesem Abschnitt werden die Mapping-Dictionaries definiert, um die
unterschiedlichen Spaltenbezeichnungen der NBA API auf ein einheitliches
Zielschema abzubilden.

- `RENAME_V2`: Mapping für `BoxScoreTraditionalV2` (historische Saisons)
- `RENAME_V3`: Mapping für `BoxScoreTraditionalV3` (aktuelle Saison)

Die Dictionaries werden einmalig definiert und in allen nachfolgenden
Verarbeitungsschritten wiederverwendet.


In [1]:
import time
import pandas as pd
from tqdm import tqdm

from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2, boxscoretraditionalv3

# Einheitliches Zielschema (Master)

# Mapping: V2 -> Master
RENAME_V2 = {
    "PLUS_MINUS": "+/-",
    "FG_PCT": "FG%",
    "FG3M": "3PM",
    "FG3A": "3PA",
    "FG3_PCT": "3P%",
    "FT_PCT": "FT%",
}

# Mapping: V3 -> Master
RENAME_V3 = {
    "minutes": "MIN",
    "fieldGoalsMade": "FGM",
    "fieldGoalsAttempted": "FGA",
    "fieldGoalsPercentage": "FG%",
    "threePointersMade": "3PM",
    "threePointersAttempted": "3PA",
    "threePointersPercentage": "3P%",
    "freeThrowsMade": "FTM",
    "freeThrowsAttempted": "FTA",
    "freeThrowsPercentage": "FT%",
    "reboundsOffensive": "OREB",
    "reboundsDefensive": "DREB",
    "reboundsTotal": "REB",
    "assists": "AST",
    "turnovers": "TOV",
    "steals": "STL",
    "blocks": "BLK",
    "foulsPersonal": "PF",
    "points": "PTS",
    "plusMinusPoints": "+/-",
}


## Daten laden
Der Datensatz wird aus einer CSV-Datei eingelesen. Die Spiel-IDs werden als Strings geladen, um führende Nullen zu erhalten.


In [2]:
df = pd.read_csv(
    r"C:\Nbaprojekt\team_traditional_2.csv",
    sep=";",
    decimal=",",
    dtype={"gameid": str}
)

df.head()
df.columns
df.dtypes


gameid     object
date       object
type       object
teamid      int64
team       object
home       object
away       object
MIN         int64
PTS         int64
FGM         int64
FGA         int64
FG%       float64
3PM         int64
3PA         int64
3P%       float64
FTM         int64
FTA         int64
FT%       float64
OREB        int64
DREB        int64
REB         int64
AST         int64
TOV       float64
STL         int64
BLK         int64
PF          int64
+/-       float64
win         int64
season      int64
dtype: object

## Spiel-ID normalisieren
Die Spiel-IDs werden auf eine einheitliche Länge von 10 Zeichen gebracht, indem führende Nullen ergänzt werden. Dazu werden die normalisierte Spiel-ID an zweite Stelle des DataFrames verschoben, um die Struktur übersichtlicher zu halten.


In [3]:
df["gameid_norm"] = df["gameid"].str.zfill(10)
col = df.pop("gameid_norm")
df.insert(1, "gameid_norm", col)

df



,gameid,gameid_norm,date,type,teamid,team,home,away,MIN,PTS,...,DREB,REB,AST,TOV,STL,BLK,PF,+/-,win,season
0,29600001,0029600001,01.11.96,regular,1610612738,BOS,BOS,CHI,48,98,...,22,36,20,18.0,10,2,33,-9.0,0,1997
1,29600001,0029600001,01.11.96,regular,1610612741,CHI,BOS,CHI,48,107,...,29,37,28,19.0,7,8,23,9.0,1,1997
2,29600002,0029600002,01.11.96,regular,1610612739,CLE,NJN,CLE,48,90,...,23,35,16,15.0,11,1,24,13.0,1,1997
3,29600002,0029600002,01.11.96,regular,1610612751,NJN,NJN,CLE,48,77,...,24,35,13,22.0,7,7,19,-13.0,0,1997
4,29600003,0029600003,01.11.96,regular,1610612749,MIL,PHI,MIL,48,111,...,31,50,21,15.0,9,7,30,8.0,1,1997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68205,42300403,0042300403,12.06.24,playoff,1610612742,DAL,DAL,BOS,48,99,...,36,43,15,9.0,5,1,17,-7.0,0,2024
68206,42300404,0042300404,14.06.24,playoff,1610612738,BOS,DAL,BOS,48,84,...,27,31,18,14.0,2,5,19,-38.0,0,2024
68207,42300404,0042300404,14.06.24,playoff,1610612742,DAL,DAL,BOS,48,122,...,39,52,21,9.0,7,2,17,38.0,1,2024
68208,42300405,0042300405,17.06.24,playoff,1610612738,BOS,BOS,DAL,48,106,...,36,51,25,9.0,9,2,15,18.0,1,2024


## Spiele der Saison 2024/25 abrufen
Über die NBA API werden alle Regular-Season-Spiele der Saison 2024/25 geladen.


In [4]:

games_2425 = leaguegamefinder.LeagueGameFinder(
    season_nullable="2024-25",
    season_type_nullable="Regular Season"
)


## Metadaten extrahieren
Die Spielübersicht wird als DataFrame gespeichert.


In [5]:
games_2425_df = games_2425.get_data_frames()[0]


## Plausibilitätscheck: Teams pro Spiel
Es wird geprüft, wie viele Zeilen pro Spiel-ID vorhanden sind.


In [6]:
games_2425_df.groupby("GAME_ID").size().value_counts().head(10)
games_2425_df


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612751,BKN,Brooklyn Nets,0022401188,2025-04-13,BKN vs. NYK,L,239,105,...,0.680,6,36,42,21,4,2,18,24,-8.0
1,22024,1610612755,PHI,Philadelphia 76ers,0022401191,2025-04-13,PHI vs. CHI,L,240,102,...,0.857,10,38,48,20,7,6,14,15,-20.0
2,22024,1610612754,IND,Indiana Pacers,0022401189,2025-04-13,IND @ CLE,W,289,126,...,0.909,14,47,61,27,15,10,13,20,8.0
3,22024,1610612762,UTA,Utah Jazz,0022401195,2025-04-13,UTA @ MIN,L,241,105,...,0.818,9,31,40,24,6,5,13,15,-11.0
4,22024,1610612765,DET,Detroit Pistons,0022401192,2025-04-13,DET @ MIL,L,264,133,...,0.643,14,28,42,35,9,3,13,19,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22024,1610612737,ATL,Atlanta Hawks,0022400064,2024-10-23,ATL vs. BKN,W,241,120,...,0.717,12,33,45,25,12,9,16,20,4.0
2456,22024,1610612752,NYK,New York Knicks,0022400061,2024-10-22,NYK @ BOS,L,241,109,...,0.750,5,29,34,20,2,3,11,12,-23.0
2457,22024,1610612738,BOS,Boston Celtics,0022400061,2024-10-22,BOS vs. NYK,W,240,132,...,0.875,11,29,40,33,6,3,3,15,23.0
2458,22024,1610612750,MIN,Minnesota Timberwolves,0022400062,2024-10-22,MIN @ LAL,L,239,103,...,0.741,12,35,47,17,4,1,15,22,-7.0


## Fehlende Spiele identifizieren
Es wird geprüft, welche Spiele der Saison 2024/25 in den vorhandenen Teamdaten noch fehlen.


In [7]:
expected_ids = set(games_2425_df["GAME_ID"].astype(str))
have_ids = set(df["gameid_norm"])

missing_2425 = sorted(expected_ids - have_ids)

len(missing_2425), missing_2425[:5]


(1230, ['0022400001', '0022400002', '0022400003', '0022400004', '0022400005'])

## Testweiser Abruf von Team-Boxscores
Für einige fehlende Spiele werden testweise die Team-Statistiken über die NBA API abgerufen, um den Boxscore-Workflow zu überprüfen.


In [8]:


test_ids = missing_2425[:3]  # nur 3 Spiele zum Test

team_parts = []
for gid in test_ids:
    bs = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=gid)
    team_df = bs.get_data_frames()[1]   # TEAM_STATS ist Index 1
    team_df["GAME_ID"] = gid
    team_parts.append(team_df)
    time.sleep(2.6)

team_test = pd.concat(team_parts, ignore_index=True)

team_test[
    ["GAME_ID","TEAM_ID","PTS","FGM","FGA","FG_PCT",
     "FG3M","FG3A","FG3_PCT","FTM","FTA","FT_PCT","PLUS_MINUS"]
].head(10)


C:\Users\Ntamb\AppData\Local\Temp\ipykernel_25736\2729628471.py:5: DeprecationWarning: BoxScoreTraditionalV2 is deprecated and will be removed in a future version. Please use BoxScoreTraditionalV3 instead. Data is no longer being published for BoxScoreTraditionalV2 as of the 2025-26 NBA season.
  bs = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=gid)


,GAME_ID,TEAM_ID,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,PLUS_MINUS
0,0022400001,1610612738,116,38,75,0.507,18,45,0.400,22,27,0.815,-1
1,0022400001,1610612737,117,50,99,0.505,10,32,0.313,7,13,0.538,1
2,0022400002,1610612748,121,47,101,0.465,17,45,0.378,10,13,0.769,-2
3,0022400002,1610612765,123,43,92,0.467,13,36,0.361,24,28,0.857,2
4,0022400003,1610612753,114,43,93,0.462,13,44,0.295,15,19,0.789,25
5,0022400003,1610612766,89,34,84,0.405,13,38,0.342,8,14,0.571,-25


## Metadaten für Merge vorbereiten
Aus der Spielübersicht werden die relevanten Metadaten extrahiert und die Spiel-IDs als Strings formatiert.


In [9]:
meta = games_2425_df[
    ["GAME_ID", "TEAM_ID", "TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "WL"]
].copy()

meta["GAME_ID"] = meta["GAME_ID"].astype(str)


## Team-Stats mit Metadaten verknüpfen
Die geladenen Team-Boxscores werden anhand von Spiel- und Team-ID mit den Spielmetadaten zusammengeführt.


In [10]:
merged = team_test.merge(
    meta,
    on=["GAME_ID", "TEAM_ID"],
    how="left"
)
merged.columns


Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION_x', 'TEAM_CITY',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', 'TEAM_ABBREVIATION_y', 'GAME_DATE', 'MATCHUP', 'WL'],
      dtype='object')

## Kontrolle des Merges
Es wird überprüft, ob Team-Stats und Metadaten korrekt zusammengeführt wurden.


In [11]:
merged[
    ["GAME_ID", "TEAM_ABBREVIATION_y", "MATCHUP", "WL", "PTS", "PLUS_MINUS"]
].head(6)


,GAME_ID,TEAM_ABBREVIATION_y,MATCHUP,WL,PTS,PLUS_MINUS
0,0022400001,BOS,BOS vs. ATL,L,116,-1
1,0022400001,ATL,ATL @ BOS,W,117,1
2,0022400002,MIA,MIA @ DET,L,121,-2
3,0022400002,DET,DET vs. MIA,W,123,2
4,0022400003,ORL,ORL vs. CHA,W,114,25
5,0022400003,CHA,CHA @ ORL,L,89,-25


## Home- und Away-Teams bestimmen
Aus dem Matchup-String werden Heim- und Auswärtsteam abgeleitet und in separate Spalten geschrieben.


In [12]:
merged["team"] = merged["TEAM_ABBREVIATION_y"]

# Gegner aus dem Matchup extrahieren (letzter Token, z.B. NYK)
opp = merged["MATCHUP"].str.split().str[-1]

# Heimspiel erkennen
is_home = merged["MATCHUP"].str.contains("vs.")

merged["home"] = merged["team"].where(is_home, opp)
merged["away"] = opp.where(is_home, merged["team"])

merged[["GAME_ID", "team", "MATCHUP", "home", "away"]].head(10)


,GAME_ID,team,MATCHUP,home,away
0,0022400001,BOS,BOS vs. ATL,BOS,ATL
1,0022400001,ATL,ATL @ BOS,BOS,ATL
2,0022400002,MIA,MIA @ DET,DET,MIA
3,0022400002,DET,DET vs. MIA,DET,MIA
4,0022400003,ORL,ORL vs. CHA,ORL,CHA
5,0022400003,CHA,CHA @ ORL,ORL,CHA


## Zusätzliche Variablen erstellen und Spalten umbenennen
Datum, Win-Flag und saisonbezogene Metadaten werden ergänzt. Anschließend werden zentrale Statistikspalten ins Zielschema umbenannt.


In [13]:


# Datum formatieren
merged["date"] = pd.to_datetime(merged["GAME_DATE"]).dt.strftime("%d.%m.%y")

# Win-Flag
merged["win"] = (merged["WL"] == "W").astype(int)

# Fixe Felder
merged["type"] = "regular"
merged["season"] = 2025  # weil 2024-25 in der CSV-Logik als 2025 geführt wird

# IDs ins Zielschema
merged["gameid"] = merged["GAME_ID"]
merged["teamid"] = merged["TEAM_ID"]

# Turnover
merged = merged.rename(columns={"TO": "TOV"})



merged = merged.rename(columns=RENAME_V2)


## Zielspalten auswählen
Es werden die relevanten Variablen in der gewünschten Reihenfolge ausgewählt und in einen finalen Test-DataFrame übernommen.


In [14]:
cols = [
    "gameid","date","type","teamid","team","home","away",
    "MIN","PTS","FGM","FGA","FG%","3PM","3PA","3P%",
    "FTM","FTA","FT%","OREB","DREB","REB","AST","TOV",
    "STL","BLK","PF","+/-","win","season"
]

final_test = merged[cols].copy()


## Team-Minuten normalisieren
Die Team-Gesamtminuten (z. B. "240:00") werden in Spielminuten umgerechnet, indem durch fünf Spieler geteilt wird.


In [15]:
min_minutes = merged["MIN"].str.split(":").str[0].astype(int)
merged["MIN"] = (min_minutes / 5).astype(int)


## Metadaten für vollständigen Abruf vorbereiten
Die relevanten Spielmetadaten der Saison 2024/25 werden erneut extrahiert und für den weiteren Boxscore-Abruf vorbereitet.


In [16]:


meta = games_2425_df[
    ["GAME_ID", "TEAM_ID", "TEAM_ABBREVIATION", "GAME_DATE", "MATCHUP", "WL"]
].copy()

meta["GAME_ID"] = meta["GAME_ID"].astype(str)


## Metadaten für Regular Season und Playoffs laden
Über die NBA API werden die Spielmetadaten für Regular Season und Playoffs der Saison 2024/25 abgerufen und in ein einheitliches Format gebracht.


In [17]:

SEASON_STR = "2024-25"
SEASON_INT = 2025  # Sowie in der csv-Logik

def get_meta(season_type: str) -> pd.DataFrame:  #Aufruf der Spiele-metadaten für 2025
    lgf = leaguegamefinder.LeagueGameFinder(
        season_nullable=SEASON_STR,
        season_type_nullable=season_type
    )
    gdf = lgf.get_data_frames()[0].copy()

    meta = gdf[["GAME_ID","TEAM_ID","TEAM_ABBREVIATION","GAME_DATE","MATCHUP","WL"]].copy()
    meta["GAME_ID"] = meta["GAME_ID"].astype(str)
    meta["TEAM_ID"] = meta["TEAM_ID"].astype(str)
    meta["GAME_DATE"] = pd.to_datetime(meta["GAME_DATE"], errors="coerce")
    return meta

meta_rs = get_meta("Regular Season")
meta_po = get_meta("Playoffs")

print("meta_rs rows:", len(meta_rs), "games:", meta_rs["GAME_ID"].nunique())
print("meta_po rows:", len(meta_po), "games:", meta_po["GAME_ID"].nunique())


meta_rs rows: 2460 games: 1230
meta_po rows: 168 games: 84


## Fehlende Spiele für Regular Season und Playoffs bestimmen
Die vorhandenen Spiel-IDs werden mit den erwarteten IDs aus Regular Season und Playoffs verglichen, um fehlende Spiele zu identifizieren.


In [18]:
have_ids = set(df["gameid_norm"].astype(str))

rs_expected = set(meta_rs["GAME_ID"].astype(str))
po_expected = set(meta_po["GAME_ID"].astype(str))

rs_missing = sorted(rs_expected - have_ids)
po_missing = sorted(po_expected - have_ids)

print("RS missing:", len(rs_missing), rs_missing[:5])
print("PO missing:", len(po_missing), po_missing[:5])


RS missing: 1230 ['0022400001', '0022400002', '0022400003', '0022400004', '0022400005']
PO missing: 84 ['0042400101', '0042400102', '0042400103', '0042400104', '0042400111']


## Hilfsfunktionen für das Daten-Mapping
Die folgenden Funktionen sorgen für ein stabiles Schema nach dem Merge (Team-Abkürzung auswählen), leiten Home/Away aus dem Matchup ab und normalisieren Team-Minuten in Spielminuten.


In [19]:

def pick_team_abbr(m: pd.DataFrame) -> pd.Series:
    # nach dem merge kann es TEAM_ABBREVIATION, _x oder _y geben
    if "TEAM_ABBREVIATION_y" in m.columns:
        return m["TEAM_ABBREVIATION_y"].astype(str)
    if "TEAM_ABBREVIATION" in m.columns:
        return m["TEAM_ABBREVIATION"].astype(str)
    return m["TEAM_ABBREVIATION_x"].astype(str)

def add_home_away(m: pd.DataFrame) -> pd.DataFrame:
    m = m.copy()
    m["team"] = pick_team_abbr(m)

    opp = m["MATCHUP"].astype(str).str.split().str[-1]
    is_home = m["MATCHUP"].astype(str).str.contains(r"vs\.", na=False)

    m["home"] = m["team"].where(is_home, opp)
    m["away"] = opp.where(is_home, m["team"])
    return m

def normalize_team_minutes(min_col: pd.Series) -> pd.Series:
    # V2: "240:00" -> 240 -> /5 -> 48
    s = min_col.astype(str).str.strip()
    team_min = pd.to_numeric(s.str.extract(r"^(\d+):")[0], errors="coerce")
    min_num = pd.to_numeric(s, errors="coerce")
    return team_min.div(5).where(team_min.notna(), min_num).round().astype("Int64")


## Fehlende Spiele nachladen (BoxScoreTraditionalV2)
Für alle fehlenden Spiel-IDs werden die Team-Boxscore-Statistiken über die NBA API (V2) geladen,
mit den Metadaten gemergt und anschließend in das einheitliche Zielschema (Spalten + Namen) gebracht.
Fehlgeschlagene Abrufe werden gesammelt und zusammen mit den erfolgreich geladenen Daten zurückgegeben.


In [ ]:

def fetch_v2_loop(missing_ids, meta_df, type_label, season_int=2025, sleep_sec=1.6):
    new_rows = []
    failed = []

    # Keys angleichen 
    meta_df = meta_df.copy()
    meta_df["GAME_ID"] = meta_df["GAME_ID"].astype(str)
    meta_df["TEAM_ID"] = meta_df["TEAM_ID"].astype(str)

    for gid in tqdm(missing_ids, desc=f"{type_label} {SEASON_STR}", unit="game"):
        ok = False

        for attempt in range(1, 6):
            try:
                bs = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=gid, timeout=80)
                team_df = bs.get_data_frames()[1].copy()   # TEAM_STATS
                team_df["GAME_ID"] = str(gid)
                ok = True
                break
            except Exception as e:
                if attempt == 5:
                    failed.append((gid, type(e).__name__, str(e)[:200]))
                time.sleep(1.2 * attempt)   # kurzer backoff

        if not ok:
            continue

        # Merge von Team-Boxscores mit Spiel-Metadaten
        team_df["TEAM_ID"] = team_df["TEAM_ID"].astype(str)
        m = team_df.merge(meta_df, on=["GAME_ID","TEAM_ID"], how="left")

        # Abgeleitete Variablen (Home/Away, Win-Indikator, Datum)
        m = add_home_away(m)
        m["win"] = (m["WL"] == "W").astype(int)
        m["date"] = pd.to_datetime(m["GAME_DATE"], errors="coerce").dt.strftime("%d.%m.%Y")

        m["type"] = type_label
        m["season"] = season_int
        m["gameid"] = m["GAME_ID"]
        m["teamid"] = m["TEAM_ID"]

        # TO -> TOV
        if "TO" in m.columns and "TOV" not in m.columns:
            m = m.rename(columns={"TO": "TOV"})

       
        m = m.rename(columns=RENAME_V2)

        # MIN normalisieren
        if "MIN" in m.columns:
            m["MIN"] = normalize_team_minutes(m["MIN"])

        # fehlende Spalten absichern (bricht sonst bei V2/V3 Mix gern)
        for c in cols:
            if c not in m.columns:
                m[c] = pd.NA

        new_rows.append(m[cols])

        time.sleep(sleep_sec)

    out = pd.concat(new_rows, ignore_index=True) if new_rows else pd.DataFrame(columns=cols)
    return out, failed



len(rs_missing): 1230
len(rs_df): 20
len(rs_failed): 0


## Regular Season und Playoffs nachladen
Die fehlenden Spiele werden getrennt für Regular Season und Playoffs abgerufen. Anschließend werden beide Teilergebnisse zu einem DataFrame zusammengeführt und die Anzahl erfolgreicher/fehlgeschlagener Abrufe geprüft. (Zum Testen werden jeweils 10 Regular/Playoff Games geholt)


In [21]:
rs_df, rs_failed = fetch_v2_loop(rs_missing[:10], meta_rs, type_label="regular", season_int=SEASON_INT, sleep_sec=1.6)
po_df, po_failed = fetch_v2_loop(po_missing[:10], meta_po, type_label="playoffs", season_int=SEASON_INT, sleep_sec=1.8)

new_2425_all = pd.concat([rs_df, po_df], ignore_index=True)

print("RS rows:", len(rs_df), "RS failed:", len(rs_failed))
print("PO rows:", len(po_df), "PO failed:", len(po_failed))
print("NEW total rows:", len(new_2425_all), "games:", new_2425_all["gameid"].nunique())


regular 2024-25:   0%|          | 0/10 [00:00<?, ?game/s]C:\Users\Ntamb\AppData\Local\Temp\ipykernel_25736\3738819484.py:15: DeprecationWarning: BoxScoreTraditionalV2 is deprecated and will be removed in a future version. Please use BoxScoreTraditionalV3 instead. Data is no longer being published for BoxScoreTraditionalV2 as of the 2025-26 NBA season.
  bs = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=gid, timeout=80)
regular 2024-25:  10%|█         | 1/10 [00:01<00:15,  1.76s/game]C:\Users\Ntamb\AppData\Local\Temp\ipykernel_25736\3738819484.py:15: DeprecationWarning: BoxScoreTraditionalV2 is deprecated and will be removed in a future version. Please use BoxScoreTraditionalV3 instead. Data is no longer being published for BoxScoreTraditionalV2 as of the 2025-26 NBA season.
  bs = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=gid, timeout=80)
regular 2024-25:  20%|██        | 2/10 [00:03<00:13,  1.73s/game]C:\Users\Ntamb\AppData\Local\Temp\ipykernel_25736\3738819484.py:15:

RS rows: 20 RS failed: 0
PO rows: 20 PO failed: 0
NEW total rows: 40 games: 20


## Neue 2024/25-Daten in Master-Datensatz integrieren
Die bestehenden Daten und die neu geladenen Spiele werden in ein gemeinsames Schema gebracht, zusammengeführt, dedupliziert (gameid, teamid) und anschließend nach Saison und Datum sortiert.


In [22]:

# Sicherstellen: gleiche Typen
df_master = df.copy()
df_master["gameid"] = df_master["gameid_norm"].astype(str)  
df_master["teamid"] = df_master["teamid"].astype(str)

new_2425_all["gameid"] = new_2425_all["gameid"].astype(str).str.zfill(10)
new_2425_all["teamid"] = new_2425_all["teamid"].astype(str)

# combine + dedupe
all_games = pd.concat([df_master.drop(columns=["gameid_norm"], errors="ignore"), new_2425_all], ignore_index=True)
all_games = all_games.drop_duplicates(subset=["gameid","teamid"])

# sortieren nach Datum
all_games["date_dt"] = pd.to_datetime(all_games["date"], dayfirst=True, errors="coerce")
all_games = all_games.sort_values(["season","date_dt","gameid","teamid"]).drop(columns=["date_dt"])

print("ALL rows:", len(all_games), "ALL games:", all_games["gameid"].nunique())
print(all_games.groupby("gameid").size().value_counts().head())


C:\Users\Ntamb\AppData\Local\Temp\ipykernel_25736\4049120413.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_games["date_dt"] = pd.to_datetime(all_games["date"], dayfirst=True, errors="coerce")


ALL rows: 68250 ALL games: 34125
2    34125
Name: count, dtype: int64


## Export des aktualisierten Datensatzes
Der zusammengeführte und bereinigte Datensatz wird als CSV-Datei gespeichert und steht für weitere Analysen zur Verfügung.


In [23]:
out_path = r"C:\Users\Ntamb\Documents\nba_merggged_up_to_2425_rs_po.csv"
all_games.to_csv(out_path, sep=";", decimal=",", index=False)
out_path


'C:\\Users\\Ntamb\\Documents\\nba_merggged_up_to_2425_rs_po.csv'

## Aktuelle Saison (2025/26) vorbereiten
Der bestehende Master-Datensatz wird geladen. Anschließend werden die Regular-Season-Spiele der Saison 2025/26 über die NBA API abgerufen und die fehlenden Game-IDs gegenüber dem Master bestimmt.


In [24]:


# bestehende Daten
all_games["gameid"] = all_games["gameid"].astype(str).str.zfill(10)
have_ids = set(all_games["gameid"].unique())

# Meta 2025-26 Regular Season
rs_2526 = leaguegamefinder.LeagueGameFinder(
    season_nullable="2025-26",
    season_type_nullable="Regular Season"
)
rs2526_df = rs_2526.get_data_frames()[0]

meta_2526 = rs2526_df[[
    "GAME_ID","TEAM_ID","TEAM_ABBREVIATION","GAME_DATE","MATCHUP","WL"
]].copy()

meta_2526["GAME_ID"] = meta_2526["GAME_ID"].astype(str)
meta_2526["TEAM_ID"] = meta_2526["TEAM_ID"].astype(str)

expected_ids = set(meta_2526["GAME_ID"])
missing_2526 = sorted(expected_ids - have_ids)

print("2026 expected games:", len(expected_ids))
print("2026 missing games:", len(missing_2526))


2026 expected games: 611
2026 missing games: 611


## Fehlende 2025/26-Spiele nachladen (V3)
Für jede fehlende Game-ID werden die Team-Boxscore-Stats über V3 abgerufen, mit den Metadaten gemergt und ins einheitliche Schema gebracht.(Hier auch nur ersten 10 Games)


In [25]:


new_rows_2526 = []
failed_2526 = []



# Meta einmal vorbereiten (nicht pro Spiel neu)
meta2 = meta_2526.rename(columns={"GAME_ID": "gameId", "TEAM_ID": "teamId"}).copy()
meta2["gameId"] = meta2["gameId"].astype(str)
meta2["teamId"] = meta2["teamId"].astype(str)

for gid in tqdm(missing_2526[:10], desc="2025-26 Regular", unit="game"):
    ok = False

    for attempt in range(1, 7):
        try:
            bs = boxscoretraditionalv3.BoxScoreTraditionalV3(game_id=gid, timeout=120)
            team_df = bs.get_data_frames()[2].copy()   # Team Stats (2 rows)
            ok = True
            break
        except Exception as e:
            if attempt == 6:
                failed_2526.append(gid)
            time.sleep(2 * attempt)

    if not ok:
        continue

    team_df["gameId"] = team_df["gameId"].astype(str)
    team_df["teamId"] = team_df["teamId"].astype(str)

    m = team_df.merge(meta2, on=["gameId","teamId"], how="left")

    # team / home / away
    m["team"] = m["TEAM_ABBREVIATION"]
    is_home = m["MATCHUP"].str.contains(r"vs\.", na=False)
    opp = m["MATCHUP"].str.split().str[-1]

    m["home"] = m["team"].where(is_home, opp)
    m["away"] = opp.where(is_home, m["team"])

    m["win"] = (m["WL"] == "W").astype(int)
    m["date"] = pd.to_datetime(m["GAME_DATE"], errors="coerce").dt.strftime("%d.%m.%Y")

    m["type"] = "regular"
    m["season"] = 2026
    m["gameid"] = m["gameId"]
    m["teamid"] = m["teamId"]

    m = m.rename(columns=RENAME_V3)

    # Minuten normalisieren (240:00 -> 48)
    s = m["MIN"].astype(str)
    team_min = pd.to_numeric(s.str.extract(r"^(\d+):")[0], errors="coerce")
    m["MIN"] = team_min.div(5).where(team_min.notna(), 48).astype("Int64")

    # fehlende Spalten absichern
    for c in cols:
        if c not in m.columns:
            m[c] = pd.NA

    new_rows_2526.append(m[cols])
    time.sleep(1.8)


2025-26 Regular: 100%|██████████| 10/10 [00:21<00:00,  2.15s/game]


## Neue 2025/26-Spiele integrieren
Die neu geladenen Spiele der Saison 2025/26 werden zusammengeführt, dedupliziert und dem bestehenden Master-Datensatz hinzugefügt. Anschließend wird die Aktualisierung überprüft.


In [26]:
season2526_games = pd.concat(new_rows_2526, ignore_index=True)

print("2026 new rows:", len(season2526_games))
print("2026 new games:", season2526_games["gameid"].nunique())
print("failed:", len(failed_2526))

# anhängen
all_games = pd.concat([all_games, season2526_games], ignore_index=True)
all_games = all_games.drop_duplicates(subset=["gameid","teamid"])

print("TOTAL games now:", all_games["gameid"].nunique())


2026 new rows: 20
2026 new games: 10
failed: 0
TOTAL games now: 34135


## Abschluss: Sortieren, Validieren und Exportieren
Der finale Master-Datensatz wird chronologisch sortiert, auf grundlegende Konsistenz geprüft (2 Teams pro Spiel) und anschließend als CSV gespeichert.


In [27]:
all_games["date_dt"] = pd.to_datetime(
    all_games["date"],
    dayfirst=True,
    errors="coerce"
)

all_games = (
    all_games
    .sort_values(["season", "date_dt", "gameid", "teamid"])
    .drop(columns=["date_dt"])
)


C:\Users\Ntamb\AppData\Local\Temp\ipykernel_25736\462187490.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_games["date_dt"] = pd.to_datetime(


In [28]:
print("ROWS:", len(all_games))
print("GAMES:", all_games["gameid"].nunique())
print("DUP (gameid, teamid):", all_games.duplicated(["gameid","teamid"]).sum())

# pro Spiel sollten 2 Teams vorhanden sein
print(all_games.groupby("gameid")["teamid"].nunique().value_counts().head())


ROWS: 68270
GAMES: 34135
DUP (gameid, teamid): 0
teamid
2    34135
Name: count, dtype: int64


In [30]:
out_path = r"C:\Users\Ntamb\Documents\nba_master_alllll_seasons_sorted.csv"
all_games.to_csv(out_path, sep=";", decimal=",", index=False)
out_path


'C:\\Users\\Ntamb\\Documents\\nba_master_alllll_seasons_sorted.csv'